# AumOS Governance in Jupyter Notebooks

This notebook demonstrates the AumOS governance magic commands provided by
`aumos_governance.jupyter_extension`. The extension adds IPython line and
cell magics that enforce static trust levels and budget limits inside a
notebook session without any adaptive or automatic adjustments.

**Design guarantees**

- Trust levels are **manual only** — set once at session start, never changed automatically.
- Budget limits are **static only** — no adaptive reallocation occurs.
- Audit logging is **recording only** — no anomaly detection.

---

In [ ]:
# Install aumos-governance from PyPI (uncomment if not already installed).
# The package requires Python 3.10+ and IPython for the magic commands to work.
#
# !pip install aumos-governance
#
# To install the optional notebook extras:
# !pip install "aumos-governance[notebook]"

In [ ]:
# Load the AumOS governance magic extension into this IPython kernel.
# After this cell runs, all %governance_* line magics become available.
%load_ext aumos_governance.jupyter_extension

In [ ]:
# Configure governance for this notebook session.
#
# --trust-level sets the static trust level (integer 0-5) for the session:
#   0 = L0_OBSERVER   (read-only observation)
#   1 = L1_MONITOR    (monitoring, no actions)
#   2 = L2_SUGGEST    (suggestions only)
#   3 = L3_ACT_APPROVE (act with approval required)
#   4 = L4_ACT_REPORT  (act and report after)
#   5 = L5_AUTONOMOUS  (full autonomy)
#
# --budget sets the total USD budget ceiling for this session.
# The budget is static — it does not auto-replenish or adjust.
#
# agent_id context: jupyter-user-01
%governance --trust-level 2 --budget 10.0

In [ ]:
# Show the current governance status for this session.
# Displays active trust level, budget ceiling, amount spent, and remaining balance.
%governance_status

## Governed Tool Execution

Use `%governance_spend` to record spending against the session budget before
or after making LLM API calls. The magic checks the projected total against
the static ceiling and denies the record if it would cause an overrun.

This is intentionally explicit — there is no automatic spend interception.
You record each cost you incur so the governance layer can enforce the limit
on subsequent calls.

In [ ]:
# Simulate a governed tool call: record the estimated cost of an LLM completion
# before making the actual API request. If the budget would be exceeded the
# call is denied and you should not proceed with the API request.
#
# Format: %governance_spend <amount_usd> --tool <tool_name>
%governance_spend 0.05 --tool gpt-4o

# At this point, if no DENIED message was printed, proceed with your API call:
# response = openai_client.chat.completions.create(...)

## Checking Budget

Call `%governance_status` at any point to inspect remaining budget. The
extension tracks cumulative spend across all `%governance_spend` calls made
since governance was enabled with `%governance`.

In [ ]:
# Display current budget status: trust level, spent, limit, remaining.
%governance_status

# Example: record a few more spend events to see the balance change.
%governance_spend 0.03 --tool text-embedding-3-small
%governance_spend 0.12 --tool claude-opus-4-6

# Check the updated balance.
%governance_status

## Audit Trail

The extension maintains a record of every governance decision made during the
session. Use `%governance_status` to confirm the final state, then
`%governance_disable` to close the session. Accumulated spend is preserved
after disabling so the final total can be reviewed.

For structured per-call audit records, use
`aumos_governance.GovernanceEngine` directly with
`GovernedOpenAIClient` or `GovernedAnthropicClient` — those wrappers
expose a typed `audit_log` list on each client instance.

In [ ]:
# Review the final governance state for this session.
%governance_status

# Disable governance enforcement when the session is complete.
# The final session total is printed for review.
%governance_disable

# For programmatic access to per-call audit records, use the engine directly:
#
# from aumos_governance import GovernanceEngine
# from aumos_governance.integrations.openai_wrapper import GovernedOpenAIClient
#
# engine = GovernanceEngine()
# client = GovernedOpenAIClient(
#     openai_client=raw_openai_client,
#     governance_engine=engine,
#     agent_id="jupyter-user-01",
#     default_cost=0.01,
# )
#
# After making calls:
# for record in client.audit_log:
#     print(record.record_id, record.call_type, record.allowed, record.estimated_cost)
#
# Engine-level audit query:
# from aumos_governance.audit.query import AuditFilter
# result = engine.audit.query(AuditFilter(agent_id="jupyter-user-01"))
# for rec in result.records:
#     print(rec.record_id, rec.outcome, rec.timestamp)